In [1]:
from lvmtipo.actors import lvm
import asyncio

In [2]:
exptime = 5.0

In [3]:
tel = lvm.sci

await tel.start()
await tel.ag.status()


[WARNING]: actor lvm.sci.pwi currently not reachable.
[WARNING]: actor lvm.sci.foc currently not reachable.
[WARNING]: actor lvm.sci.agcam currently not reachable.
[WARNING]: actor lvm.sci.km currently not reachable.
[WARNING]: actor lvm.sci.ag currently not reachable.


Exception: Failed routing message to consumer 'lvm.sci.ag'.

In [41]:
# dont forget the focus !

await tel.foc.status()

{'Reachable': True,
 'AtHome': False,
 'Moving': False,
 'PositionSwitchStatus': 254,
 'Position': 42.0,
 'DeviceEncoder': {'Position': 1000.0, 'Unit': 'STEPS'},
 'Velocity': 0.0,
 'AtLimit': False}

In [39]:
ra_h, dec_d = (await tel.pwi.status()).flatten().unpack("ra_j2000_hours", "dec_j2000_degs")
print(f"ra_h:dec_d {ra_h}:{dec_d}")

await tel.km.slewStart(ra_h, dec_d)

ra_h:dec_d 22.959229413523:-29.6242908413143


{'Reachable': True,
 'AtHome': False,
 'Moving': False,
 'PositionSwitchStatus': 254,
 'Position': -61.129777785164805,
 'DeviceEncoder': {'Position': 1329664.0, 'Unit': 'STEPS'},
 'Velocity': 0.0,
 'AtLimit': False,
 'CurrentTime': 1678460538878999.0,
 'Simulate': True,
 'SkyPA': -122.25955555703297}

In [40]:
# ra/dec are optional and currently not used
print(await tel.ag.guideStart(exptime, ra_h, dec_d))
await asyncio.sleep(30.0)
print(await tel.ag.guideStop())

{'state': 'GUIDE', 'reference_position': {'_type': 'astropy.coordinates.SkyCoord', 'frame': 'fk5', 'representation': 'spherical', 'lon': 116.0934634744157, 'lat': -9.552497968628044}, 'reference_filenames': ['/data/lvm/skyw/agcam/west/20230225/lvm.skyw.agcam.west_00000287.fits', '/data/lvm/skyw/agcam/east/20230225/lvm.skyw.agcam.east_00000287.fits']}
{'state': 'IDLE'}
